<center>
    <h1> ILI-286 - Laboratorio #2 </h1>
    <h2> Integración numérica aplicada </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Integración Impropia](#se1)
  2. [The Caputo's Fractional Derivative](#se2)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)
* [Anexos](#Anexos) $\leftarrow$ Comenzar ejeciución aquí
  * [Implementación de algoritmos de integración](#an1)
  * [Implementación de la derivada fraccionaria](#an2)

<div id='intro' />
## Introducción


<div id='desarrollo'/>
## Desarrollo y analisís de resultados

<div id='se1'/>
## 1.- Integración Impropia

<div id='de1'/>
### 1.1) Cambio de variable propuesto
Si se quiere llevar la variable $t$ desde el intervalo de integración $[-\infty,x]$ a el $[a,b]$ se hace un cambio de 
variable utilizando la nueva variable $u$ definida por
\begin{align}
    u = -\displaystyle \frac{b-a}{t-x-1} + a \quad &\Rightarrow t = \displaystyle \frac{b + a \, x + u (1+x)} {a-u} \\
    &\Rightarrow dt = \displaystyle \frac{b-a}{(u-a)^2} du
\end{align}

Obtenemos el cambio de variables dependiente de $a$, $b$ y $x$. Graficamente:

In [ ]:
#La siguiente función grafica el cambio de variables para a,b y x dados.
#Pruebe otros valores:
plot_sust1_abx(a=-1, b=1, x=3)

Aplicando la transformación a la integral dada tenemos:
$$ f(x) = \int_{-\infty}^x \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{t^2}{2}} \, dt $$

$$ f(x) = \int_{a}^b \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{{(-b-a\,x + u(1+x)})^2}{2(u-a)^2}} \, \frac{b-a}{(u-a)^2} du $$
Como podemos ver, ahora la integral depende de $a$, $b$ y $x$. 
Definamos estas funciones para su posterior utilización:

In [ ]:
#Integrales:
int_orig = lambda t: (np.e**((-t**2)/2))/ np.sqrt(2*np.pi)
def get_int_abx(a, b, x):
    return lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)

<div id='de2'/>
### 1.2) Complejidad algoritmica de los métodos  de integración
La implementación de los mismos puede analizarse en el [primer anexo](#an1).
Para cada una de las siguiente complejidades se asume una cantidad de intervalos $n$ y un costo de 
evaluación de la función de $o(f)$

##### 1.2.1) Midpoint Rule
 * Linspace: $1$ resta, $1$ división y $(n+1)$ multiplicaciones.
 * Diferencial: $1$ resta.
 * Puntos medios: $n$ multiplicaciones.
 * Evaluación: $n$ evaluaciones de la función.
 * Suma final: $n$ sumas y $1$ multiplicación.
 * **Total:** $1 + 1 + (n+1) + 1 + n + no(f) + n + 1 = 5 + 3n + no(f) = O(3n + no(f))$ 

##### 1.2.2) Trapezoid Method
 * Linspace + diferencial: $n + 4$ operaciones.
 * Calculo de la integral: $(n+1)$ evaluaciones de la función, $(n-1)$ multiplicaciones, $n+1$ 
 sumas y $2$ multiplicaciones.
 * **Total:** $n + 4 + (n+1)o(f) + (n-1) + (n+1) + 2 = 6 + 3n + (n+1)o(f) = O(3n + (n+1)o(f))$

##### 1.2.3) Simpson's Rule
 * Linspace + Diferencial:  $n+4$ operaciones.
 * Calculo de la integral: 
     * (n/2) evaluaciones en cada lado (derecha, centro y izquierda): 3(n/2) evaluaciones de $f$.
     * (n/2) multiplicaciones (del centro).
     * (n/2) multiplicaciones y una divición.
     * (n/2) sumas.
 * **Total:** $n+4 + \frac{3n}{2}o(f) +\frac{n}{2} + \frac{n}{2} +1 + \frac{n}{2} 
 = 5 + \frac{5n}{2} + \frac{3n}{2}o(f) = O(\frac{5n}{2} + \frac{3n}{2}o(f))$

##### 1.2.4) Gaussian Quadrature
 * Calculo de la integral: $n$ evaluaciones $n$ multiplicaciones y $n$ sumas: $no(f) + 2n$
 * La complejidad de la función que encuentra los puntos y sus pesos está determinada por el
 uso de `numpy.linalg.eigh` para una matriz de $n\times n$. Según la documentación[[1]](#ref1) 
 `eigh` utiliza las rutinas *LAPACK* `_syevd` y `_heevd` por lo que su complejidad debe ser 
 $O(n^3)$[[2]](#ref2).
 * **Total:** $2n + no(f) + n^3 = O(no(f) + n^3)$

<div id='de3'/>
### 1.3) Análisis de los métodos de integración
Para poder comparar los diferentes metodos de integración es necesario primero obtener la data:

In [ ]:
a = -1
b =  1
Ns = [10, 30, 100, 250] #, 500, 1000, 2000]

#Inicializa la forma de almacenar la data: data[metodo][h] 0: alpha, 1: tiempo, 2: resultados.
data = {}
for name in method:
    data[name] = {}
    for n in Ns:
        data[name][n] = [[], [], []]

#Función para guardar los resultados obtenidos
def save_result(name, f, n, a, b):
    data[name][n][2][-1] = method[name](f, n, a,b)

#Bucle de recolección de data.
for name in method:
    for alpha in range(0,50):
        func = get_int_abx(a,b, 10**alpha)
        #No se puede hacer timeit -q -o (Bug#8864) solucionado en Iphyton 4.1 :(
        for n in Ns:
            data[name][n][2].append( None ) # Para evitar que timeit genere más de lo necesario
            t = %timeit -n 1 -r 1 -o save_result(name, func, n, a+delta, b-delta)
            data[name][n][0].append( alpha )
            data[name][n][1].append( t.best )
        #Borrando el output innecesario.
        clear_output()

In [ ]:
#Gráficas de los tiempos para cada metodo.
plot_time_h(data, Ns)

Como podemos notar el tiempo solo depende de los valores de $h$ escogidos, por lo tanto,
para comparar los métodos calcularemos la media del tiempo para cada $h$ obteniendo el siguiente
gráfico:

In [ ]:
#Gráfico que compara los tiempos de ejecución media de los metodos dependiendo de h
plot_comp_methods(data, Ns)

Podemos notar claramente como el método de la cuadratura Gaussiana es el más lento, esto 
se debe a que obtiene soluciones de mayor calidad por medio de procesos más costosos 
(operaciones matriciales), y por ello su tiempo de ejecución
crece exponencialmente a $h$.

Por otro lado, los demás métodos se comportan con un crecimiento más bien lineal y con
tiempos de ejecución más cercanos entre si.

In [ ]:
#Gráficos de los resultados dependiendo de alpha.
plot_result_h(data, Ns)

<div id='de4'/>
### 1.4) Caso $x = 0$
Sabemos que en $x=0$ deberíamos obtener $0.5$ pues estamos integrando la mitad de una función de probabilidad
simetrica.

In [ ]:
a = -1
b =  1
N =  500
func3 = get_int_abx(a, b, x=0)
for name in method:
    try:     print(name, "obtuvo:", method[name](func3, N, a,b))
    except:  print(name, "falló!")

Podemos observar como los métodos de Simpson y del trapecio fallaron debido a que en este caso
no podemos calcular los extremos. Aún así obtenemos el resultado correcto con el método del 
punto medio y la cuadratura Gaussiana, esta última sin error.

Para los métodos que fallarón estimaremos la integral restado un número muy pequeño
($\delta = 10^{-15}$) en los extremos:

In [ ]:
print(simpsons( func3, N, a-delta, b-delta))
print(trapezoid(func3, N, a-delta, b-delta))

<div id='de5'/>
### 1.5) Caso $ x = +\infty$
Si se quiere llevar la variable $t$ desde el intervalo de integración $[-\infty,\infty]$ a el $[a,b]$ 
se hace un cambio de variable utilizando la nueva variable $u$ definida por:
\begin{align}
   u = \displaystyle \frac{b-a}{\pi} \, \tan^{-1} (t) + \displaystyle \frac{b-a}{2} +a \quad 
   & \Rightarrow  t = \displaystyle \cot ( \frac {\pi (u-b)}{a-b}) \\
   & \Rightarrow dt = - \displaystyle \frac {\pi \csc^2 ( \displaystyle \frac {\pi (u-b)}{a-b} )} {a-b} \, du
\end{align}
Obtenemos el cambio de variables dependiente de $a$ y $b$, gráficamente:

In [ ]:
#La siguiente función grafica el cambio de variables para a y b dados.
#Puede probar otros valores:
plot_sust2_ab(a=-1, b=1)

Reemplazando en la integral nos queda:
$$ \lim_{x\rightarrow \infty}f(x) = \int_{-\infty}^\infty \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{t^2}{2}} \, dt $$

$$ = \int_a^b \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{- \displaystyle \frac{1}{2} \, \displaystyle \cot^2 ( \frac {\pi (u-b)}{a-b})}  \displaystyle \frac {-\pi \csc^2 ( \displaystyle \frac {\pi (u-b)}{a-b} )} {a-b} \, du  $$

Utilizando este cambio de variables para calcular la integral tenemos:

In [ ]:
N = 50
a = -1
b =  1

func_inf_inf = lambda u: np.e**(-((  np.tan(np.pi * (u-b)/(a-b)  )**-2 )/2)) * (  (-np.pi *(np.sin(np.pi * (u-b)/(a-b)))**-2)/(a-b)  )/ np.sqrt(2*np.pi)

print(    midpoint(func_inf_inf, N, a,       b))
print(   trapezoid(func_inf_inf, N, a+delta, b-delta))
print(    simpsons(func_inf_inf, N, a+delta, b-delta))
print(gaussianquad(func_inf_inf, N, a,       b))

Podemos ver que todos los métodos dan resultados muy cercanos a $1$ que debería ser la 
respuesta de integrar una función de densidad de probabilidad en todo su dominio. 
El método de punto medio y la cuadratura Gaussiana no necesitaron ninguna modificación 
y tienen errores pequeños. Por otro lado en el método de Simpson y en el del trapecio se 
integró en un intervalo ligeramente más pequeño para evitar los errores que daría integrar
en los extremos.

<div id='se2'/>
## 2.- Derivación fraccionaria

<div id='de6'/>
### 2.1- Cambio de variable
Si se quiere llevar la variable $\tau$ desde el intervalo de integración $[a,t]$ a el $[-1,1]$ se hace un cambio de 
variable utilizando la nueva variable $u$ definida por
\begin{align}
    u = \displaystyle \frac{a+t-2 \, \tau}{a-t}
    & \Rightarrow   \tau = \displaystyle \frac{a \, (1-u) + t \, (1+u)} {2} \\
    & \Rightarrow d \tau = \displaystyle \frac{t-a}{2} du
\end{align}
Obtenemos un cambio de variables dependiente de $a$ y $\tau$. Graficamente:

In [ ]:
#La siguiente función grafica el cambio de variables para a y t (tau) dados.
#Puede probar otros valores:
plot_sust3_at(a=0, t=2)

Aplicando la transformación a la integral dada tenemos:
$$ \int_{a}^t \displaystyle \frac {f'(\tau)}{(t-\tau)^\alpha} \, d\tau $$

$$ = \int_{-1}^1 \displaystyle \frac {f'(\displaystyle \frac{a \, (1-u) + t \, (1+u)} {2})}{(t-\displaystyle \frac{a \, (1-u) + t \, (1+u)} {2})^\alpha} \, \displaystyle \frac{t-a}{2} du $$
Como podemos ver, ahora la integral depende de $a$, $\tau$ y $\alpha$. 
Definamos estas funciones para su posterior utilización:

In [ ]:
#Cálculo de la integral de la derivada fraccionaria, devuelve una función que depende te de t.
t = sp.Symbol('t', real=True, positive=True, imaginary=False)
u = sp.Symbol('u', real=True, positive=True, imaginary=False)
#establecemos la cantidad de intervalos a utilizar en cuadratura gaussiana
N = 50
def get_int_df(fun, a, alpha):
    #diferenciamos una vez la funcion recibida
    g = sp.diff(fun, t);
    #calcularemos lo que esta dentro de la integral
    #evaluamos la funcion diferenciada en el cambio de variable que correspondia a Tau,
    #luego lo dividimos por el denominador de dentro de la integral y lo multiplicamos
    #por el factor que acompaña a du.
    g_eval = (g.subs(t, ((a*(1-u)+t*(1+u))/sp.Integer(2))) / (t- (a*(1-u)+t*(1+u))/sp.Integer(2)  )**alpha) * (t-a)/sp.Integer(2)
    
    #una vez tenemos nuestra funcion dentro de la integral que depende de u la integramos entre -1 y 1 gracias a cambio
    #de variable utilizando cuadratura gaussiana con funciones simbolicas de simpy
    f_int = gaussianquad_sympy(g_eval, N, -1, 1)
    
    #tomamos el resultado real, con la raíz positiva
    f_int = f_int.subs(sp.zoo, 2)
    return f_int

<div id='de7'/>
### 2.2- Implementación de fraction_diff
#TODO: Faltan comentarios de la implementación, debería poner el código en anexos.

In [ ]:
t = sp.Symbol('t', positive=True, real=True, imaginary=False)
def fraction_diff(f, t0, alpha, a=0):
    tau = sp.Symbol('Tau', positive = True, real=True, imaginary=False)
    #calculamos el valor de n como el piso de alpha
    n = sp.Integer(sp.floor(alpha))
    #calculamos el valor del nuevo alpha restandole n y lo almacenamos en frac
    frac = sp.Rational(alpha - n,1)
    
    fr_diff = f
    
    #si n es dinstinto de cero entonces derivamos la función n veces
    if (n != 0):
        fr_diff = sp.diff(f, t, n)
        
    #una vez derivada n veces la función procedemos a realizar la derivada fraccionaria siempre que frac
    #no sea 0
    if (frac != 0):
        fr_diff = get_int_df(fr_diff, a, frac)
    
    #dividimos la derivada fraccionaria por la función gamma
    fr_diff = fr_diff/sp.gamma(1-frac)
    #evaluamos en t0
    return fr_diff.subs(t, t0)

<div id='de8'/>
### 2.3- Cálculos

In [ ]:
t = sp.Symbol('t', positive=True, real=True, imaginary=False) #t ahora es una variable simbolica.
r = sp.Rational #r(a,b) retornará: a/b simbolico, no calcula en número.
functions_and_data = [
#   |   f(t)    |     t     |   alpha     |
    ( 20        , r( 3, 10) , r(  2, 10) ),
    ( sp.sin(t) , r( 2,  1) , r( 15, 10) ),
    ( sp.exp(t) , r(50,  1) , r(464, 10) ),
    ( t         , r( 8, 10) , r(  7, 10) ),
    ( sp.log(t) , r( 4, 10) , r(  3, 10) ),
    ( t**2      , r( 3,  1) , r(  5, 10 )),
    (sp.tanh(t) , r( 3,  1) , r(  5, 10) )
] 

for f in functions_and_data:
    style_math(*f, res=fraction_diff(*f, a=0).evalf() )

<div id='de9'/>
### 2.3- Análisis de comportamiento:

In [ ]:
func_min_max = [(f, int(a), int(a)+1) for f, _, a in functions_and_data]

#Cuidado! no siempre se cumple que alpha < t, ojo con el log
for data in func_min_max:
    #if (str(data[0]) == "t"): #TODO: Error con el logaritmo. "t"): #TODO: Error con el logaritmo.
    print("Para f(t)=", str(data[0]))
    plot_0_2pi(*data, evals=8, points=200)

En los gráficos podemos ver como las derivadas fracionarias son *pasos intermedios* entre dos derivadas enteras.
Los mejores ejemplos de esto son los gráficos de $f(t) = t$, $f(t) = t^2$ y $f(t) = \tanh{(t)}$ donde se puede notar
como cada aproximación hace que la curva se asemeje más a la derivada entera más cercana.

## Conclusiones

## Referencias
<div id='ref1'\> [1] [NumPy v1.10 Manual](http://docs.scipy.org/doc/numpy-1.10.1/reference/generated/numpy.linalg.eigh.html) numpy.linalg.eigh. *The eigenvalues/eigenvectors are computed using LAPACK routines _syevd, _heevd*. Revisado 22/10/2015

<div id='ref2'\> [2] [LAPACK Benchmark](http://www.netlib.org/lapack/lug/node71.html). Ver tabla 3.13. Revisado el 22/10/2015

## Anexos
Comenzamos cargando los modulos necesarios:

In [ ]:
import matplotlib
%matplotlib inline
import numpy as np
import sympy as sp
import scipy as sy
import matplotlib.pyplot as plt
#Para retornas formulas matemáticas
from IPython.display import display, Math, Latex
#Para transformar las funciones sympy a lambdas
from sympy.utilities.lambdify import lambdify
#Para borrar el output de timeit, quitar en ipython 4.1 y ejecutar timeit con -q.
from IPython.display import clear_output
#Tamaño de los graficos
FIGSIZE = (12, 6)
#Número muy pequeño:
delta = 10**-15
#IGNORAR WARNINGS
import warnings
warnings.filterwarnings('ignore')

<div id='an1'\> 
### Implementación de algoritmos de integración:
Las siguientes funciones reciben como parámetros de entrada una función $f(x)$ en la variable `func`, un número de intervalos `N` y los números `a` y `b` que serán el rango de integración.

Retornan la estimación numerica de $\int_a^b f(x)\,dx$ mediante el método seleccionado.
#### Midpoint Rule

In [ ]:
def midpoint(fun, N, a, b):
    #Vectorizamos la funcion para poder llamarla con muchos puntos a la vez.
    f_vec = np.vectorize(fun)
    #Obtenemos N intervalos y N+1 puntos equidistantes entre a y b.
    x = np.linspace(a, b, N+1)
    #Obtenemos el diferencial de distancia entre los puntos.
    dx = x[1]-x[0]
    #Obtenemos los puntos medios del intervalo.
    midpoints = x[:-1] + 0.5*dx
    #Evaluamos en los puntos
    midpoint_values = f_vec(midpoints)
    #Calculamos y retornamos el resultado final
    return sum(midpoint_values*dx)

#### Trapezoid Method

In [ ]:
def trapezoid(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x     = np.linspace(a, b, N+1)
    dx    = x[1]-x[0]
    xmiddle = x[1:-1] #Los puntos que no estan a los extremos del intervalo.
    return 0.5*dx*(f_vec(x[0])+2*sum(f_vec(xmiddle))+f_vec(x[-1]))

#### Simpson's Rule

In [ ]:
def simpsons(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x     = np.linspace(a, b, N+1)
    if N%2==1:
        print("Simpsons solo aplica con una cantidad de intervalos pares.")
        return
    dx = x[1]-x[0]
    #Obtenemos elementos de la izquierda partiendo desde el primero de dos en dos hasta el antepenultimo.
    xleft   = x[:-2:2]
    #Obtenemos elementos centrales partiendo desde el segundo de dos en dos.
    xmiddle = x[1::2]
    #Obtenemos elementos de la derecha partiendo desde el tercero de dos en dos.
    xright  = x[2::2]
    #Los sumamos y multiplicamos. La multiplicación por 2 es implicita pues xleft y xright contienen los 
    # mismos elementos menos el primero y el último.
    return sum((f_vec(xleft)+4*f_vec(xmiddle)+f_vec(xright))*(dx/3))

#### Gaussian Quadrature

In [ ]:
def gaussianquad(fun, N, a, b):
    f_vec = np.vectorize(fun)
    #Calculamos los pesos y los nodos por el método visto en el ipython 'U2_QuadWorld.ipynb'
    x, w = gaussian_nodes_and_weights(N, a, b)
    return sum( w * f_vec(x) )

# Función gaussiana idéntica a gaussianquad solo que utiliza sintaxis de sympy para utilizarse 
# con funciones de sympy en vez de funciones lambda
def gaussianquad_sympy(fun, N, a, b):
    x, w = gaussian_nodes_and_weights(N, a, b)
    val = 0;
    for i in range(0, x.size):
        val = val + w[i]*fun.subs(u, x[i])
    return val

def gaussian_nodes_and_weights(N, a, b):
    if N==1: 
        return np.array([1]), np.array([2])
    beta = .5 / np.sqrt(1.-(2.*np.arange(1.,N))**(-2))
    T = np.diag(beta,1) + np.diag(beta,-1)
    D, V = np.linalg.eigh(T)
    x = D
    #Reescalamiento -1, 1
    x = .5 * ( (b-a)*x + b + a)
    w = 2*V[0,:]**2
    w = .5*(b-a)*w
    return x, w

<div id='an2'\> 
### Implementación de la derivada fraccionaria

### Utilidades

In [ ]:
method = {
    "Punto Medio": midpoint,
    "Trapecio": trapezoid,
    "Simpson": simpsons,
    "Cuadratura Gaussiana": gaussianquad
}

#### Gráficas:
Para el [cambio de variables 1](#de1):

In [ ]:
def plot_sust1_abx(a, b, x):
    plt.figure(figsize=FIGSIZE)
    t = np.linspace(-16*x, x, 300)
    plt.plot(t, -(b-a)/(t-x-1)+a)
    plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
    plt.xlabel("variable t")
    plt.ylabel("variable u")
    plt.show()

Para el [cambio de variables 2](#de5):

In [ ]:
def plot_sust2_ab(a, b):
    plt.figure(figsize=FIGSIZE)
    x = np.inf;
    t = np.linspace(-10, 10, 200)
    plt.plot(t, ((b-a)*np.arctan(t))/np.pi + (b-a)/2 + a)
    plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
    plt.xlabel("variable t")
    plt.ylabel("variable u")
    plt.show()

Para el [cambio de variables 3](#de6):

In [ ]:
def plot_sust3_at(a, t):
    plt.figure(figsize=FIGSIZE)
    T = np.linspace(a, t, 200)
    plt.plot(T, (a+t-2*T)/(a-t))
    plt.title('Cambio de variable asumiendo a='+str(a) + " t=" + str(t))
    plt.xlabel("variable Tau")
    plt.ylabel("variable u")
    plt.show()

Graficas [comparativas de los métodos de integración](#de3):

In [ ]:
#Muestra un gráfico de alpha vs tiempo para cada metodo comparando el h.
def plot_time_h(data, Hs):
    for name in data:
        plt.figure(figsize=FIGSIZE)
        for h in Hs:
            plt.plot(data[name][h][0], data[name][h][1], label="h = " + str(h))
            plt.title('Tiempos de ejecución de ' + name)
            plt.ylabel("Tiempo de ejecución [s]")
            plt.xlabel(r"valor de $\alpha$")
            plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
        plt.show()

In [ ]:
#Muestra un gráfico de h vs tiempo medio, comparando los metodos.
def plot_comp_methods(data, Hs):
    plt.figure(figsize=FIGSIZE)
    for name in data:
        m_time = []
        for h in Hs:
            m_time.append( sum(data[name][h][1])/len(data[name][h][1]) )
        plt.plot(Hs, m_time, label=name)
        plt.title("Comparación de tiempos de ejecución")
        plt.ylabel("Tiempo de ejecución [s]")
        plt.xlabel(r"valor de h$")
        plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
    plt.show()

In [ ]:
#Muestra un gráfico de alpha vs resultado para cada metodo comparando el h.
def plot_result_h(data, Hs):
    for name in data:
        plt.figure(figsize=FIGSIZE)
        for h in Hs:
            plt.plot(data[name][h][0], data[name][h][2], label="h = " + str(h))
            plt.title('Resultados de ' + name)
            plt.ylabel("resultado")
            plt.xlabel(r"valor de $\alpha$")
            plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
        plt.show()

Para el retorno de datos en los [calculos de derivadas fracionarias](#de8):

In [ ]:
#Muestra los resultados con estilo matematico:
def style_math(func, t, alpha, res):
    if "**" in str(func): func = str(func).replace("**", "^") #fix pow
    latex_code = r'\frac{\partial^{%s} %s}{\partial t^{%s}} \Big|_{t=%s} = %s' % (alpha, func, alpha, t, res)
    display(Math(latex_code))

Para los gráficos del [analisis de derivación fracional](#de9) en $[0, 2\pi]$

In [ ]:
def plot_0_2pi(func, alph_min, alph_max, evals, points=20):
    #graficamos desde 10^-10 hasta 2*Pi, no usamos 0 ya que la derivada 
    #fraccionaria de logaritmo entrega número complejos que nos dan problemas al graficar
    interval_0_2pi = np.linspace(10**-10, 2*np.pi, points)
    plt.figure(figsize=FIGSIZE)
    plt.tick_params(axis='x', which='major', labelsize=20)
    t  = sp.Symbol('t', positive=True, real=True, imaginary=False)
    alphs = np.linspace(alph_min, alph_max, evals)
    for alph in alphs:
        label = r'$\frac{\partial^{%.3f} %s}{\partial t^{%.3f}}$' % (alph, func, alph)
        if type(func) == type(1): #No calcular cuando la derivada es 0
            eval_func = lambda x: 0*x
        elif(alph == 0.0): #No calcular cuando no se debe derivar
            #Para log da una linea en 0 o.o?
            eval_func = lambdify(t, func, modules=['numpy','sympy'])
        else:
            frac_func = fraction_diff(func, t, alph)
            eval_func = np.vectorize(lambdify(t, frac_func, modules=['numpy','sympy']))
        try:
            plt.plot(interval_0_2pi, eval_func(interval_0_2pi), label=label)
        except ValueError:
            #Pasa cuando deriva una constante o una variable por si misma, no da 1 para dx/dx
            print("Algo paso cuando alpha=%.3f" % alph )

    plt.xticks([0, np.pi/2, np.pi, 3*np.pi/2, 2*np.pi],
               ['$0$', r'$\frac{\pi}{2}$', r'$\pi$', r'$\frac{3\pi}{2}$', r'$2\pi$'])
    plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., prop={'size':20})
    plt.show()